In [1]:
import mxnet as mx
import logging
import numpy as np
import os, urllib2, time
import warnings
warnings.simplefilter('ignore',DeprecationWarning)
logging.basicConfig(level=logging.DEBUG)

In [2]:
def download(url,filename):
    if not os.path.exists(filename):
        # urllib.urlretrieve(url, filename)
        resp = urllib2.urlopen(url)
        with open(filename, 'wb') as f:
            f.write(resp.read())

In [3]:
import pickle as pkl
download('https://s3-us-west-1.amazonaws.com/nervana-course/imdb_data.pkl','imdb_data')
data = pkl.load(open('imdb_data', 'r'))

In [4]:
data

{'X_train': array([[    0,     0,     0, ...,  1667,  3700,  1667],
        [    8,   982,   163, ...,   868,   159,   307],
        [  966,     8,    33, ...,    14,  5196,    24],
        ..., 
        [ 2128,  1862,     4, ...,    19,    14,   278],
        [  150,     8,    79, ...,     6, 10713,   353],
        [   37,   282,   151, ...,     6,   341,   163]], dtype=int32),
 'X_valid': array([[    0,     0,     0, ...,  4644,    16,  3464],
        [    4,    22,    11, ...,  1493,    11,    36],
        [ 2119,    16,   691, ...,    45,  3966,  3336],
        ..., 
        [   12,   117,   765, ...,  2513,    14,    36],
        [   47,     6,   227, ...,     3,  1688, 14829],
        [  290,     7,     2, ...,     5,  2430,   973]], dtype=int32),
 'Y_train': array([[1],
        [0],
        [0],
        ..., 
        [0],
        [0],
        [0]], dtype=int32),
 'Y_valid': array([[1],
        [1],
        [1],
        ..., 
        [1],
        [0],
        [1]], dtype=int32)}

In [5]:
#data['Y_train'] = [y[0] for y in data['Y_train']]

In [6]:
import numpy as np
from numpy import newaxis

data['X_train'] = mx.nd.array(data['X_train'], dtype=np.int32)
data['X_valid'] = mx.nd.array(data['X_valid'], dtype=np.int32)
data_x_train = data['X_train'].reshape((-1,128))
data_x_valid = data['X_valid'].reshape((-1,128))
data['Y_train'] = mx.nd.array(data['Y_train'])
data['Y_valid'] = mx.nd.array(data['Y_valid'])
data_y_train = data['Y_train'] #.reshape((-1, 2))
data_y_valid = data['Y_valid'] #.reshape((-1, 2))
train_set = mx.io.NDArrayIter(data_x_train, data_y_train, batch_size=32)
valid_set = mx.io.NDArrayIter(data_x_valid, data_y_valid, batch_size=32)

In [7]:
data['X_train'].shape

(20009L, 128L)

In [8]:
print train_set.provide_label
print train_set.provide_data

[DataDesc[softmax_label,(32, 1L),<type 'numpy.float32'>,NCHW]]
[DataDesc[data,(32, 128L),<type 'numpy.int32'>,NCHW]]


In [9]:
stack = mx.rnn.SequentialRNNCell()
stack.add(mx.rnn.LSTMCell(num_hidden=64, prefix='lstm_1'))


In [10]:
def sym_gen():
    data = mx.sym.Variable('data')
    label = mx.sym.Variable('softmax_label')
    embed = mx.sym.Embedding(data=data, input_dim=20000,output_dim=56, name='embed')
    stack.reset()
    outputs, states = stack.unroll(128, inputs=embed, merge_outputs=True)
    output = mx.sym.Dropout(data=outputs,p=0.5)
    pred = mx.sym.Reshape(output, shape=(-1, 64*128))
    pred = mx.sym.FullyConnected(data=pred, num_hidden=2, name='pred')
    label = mx.sym.Reshape(label, shape=(-1,))
    pred = mx.sym.SoftmaxOutput(data=pred, label=label, name='softmax')
    return pred

In [11]:
ctx = mx.cpu(0)

In [12]:
model = mx.mod.Module(sym_gen(), context=ctx)

In [13]:
from mxnet.notebook.callback import LiveLearningCurve
cb_args = LiveLearningCurve(metric_name='cross-entropy',frequent=10,display_freq=5).callback_args()

Loading BokehJS ...

In [14]:
cb_args

{'batch_end_callback': <bound method LiveLearningCurve.batch_cb of <mxnet.notebook.callback.LiveLearningCurve object at 0x7f5f511196d0>>,
 'eval_end_callback': <bound method LiveLearningCurve.eval_cb of <mxnet.notebook.callback.LiveLearningCurve object at 0x7f5f511196d0>>}

In [15]:
model.fit(
        train_data          = train_set,
        eval_data           = valid_set,
        eval_metric         = mx.metric.CrossEntropy(),
        optimizer           = mx.optimizer.AdaGrad(),
        optimizer_params    = {'learning_rate': 0.0002},
        initializer         = mx.init.Xavier(),
        num_epoch           = 2,
        batch_end_callback  = cb_args['batch_end_callback'])

/usr/local/lib/python2.7/dist-packages/mxnet-0.9.5-py2.7.egg/mxnet/module/base_module.py:465: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.03125). Is this intended?
  optimizer_params=optimizer_params)
INFO:root:Epoch[0] Train-cross-entropy=0.293430
INFO:root:Epoch[0] Time cost=63.336
INFO:root:Epoch[0] Validation-cross-entropy=0.364684
INFO:root:Epoch[1] Train-cross-entropy=0.201846
INFO:root:Epoch[1] Time cost=61.222
INFO:root:Epoch[1] Validation-cross-entropy=0.442820


In [16]:
model

In [17]:
model.score(valid_set, mx.metric.Accuracy())

[('accuracy', 0.8645833333333334)]